Demonstrates the usage of InferExcept for overriding the inferred schema properties.

In [31]:
# Set up the Django environment for the example
from examples.django_setup import setup_django_environment

setup_django_environment()

Assuming the following Django model:

In [32]:
from typing import final

from django.db import models


@final
class SomeRelatedModel(models.Model):
    base_field = models.CharField(max_length=100, help_text="Other Name")

    @final
    class Meta:
        app_label = "tests"

@final
class SomeModel(models.Model):
    id = models.AutoField(primary_key=True)
    some_field = models.CharField(max_length=100, help_text="Name")
    related_field = models.ForeignKey(SomeRelatedModel, on_delete=models.CASCADE, verbose_name="Verbose")

    @final
    class Meta:
        app_label = "tests"


Assuming the following Pydantic model:

In [33]:
from django2pydantic import BaseSchema, InferExcept, SchemaConfig
from django2pydantic.types import Infer

CUSTOM_TITLE = "Custom Title"
CUSTOM_DESCRIPTION = "Overridden Name"
CUSTOM_SCHEMA_EXT= "x-ngc-extension"
CUSTOM_SCHEMA_VALUE = "ngc"

class SomeSchema(BaseSchema[SomeModel]):
    """Book schema example with nested fields."""

    config = SchemaConfig[SomeModel](
        model=SomeModel,
        fields={
            "id": Infer,
            "some_field": InferExcept(
                description=CUSTOM_DESCRIPTION,
                title=CUSTOM_TITLE
                ),
            "related_field": InferExcept(
                json_schema_extra={CUSTOM_SCHEMA_EXT: CUSTOM_SCHEMA_VALUE}
            ),

        }
    )

The following schema will be produced:

In [34]:
from rich import print_json

print_json(data=SomeSchema.model_json_schema())

{
  "properties": {
    "id": {
      "description": "id",
      "maximum": 9223372036854775807,
      "minimum": -9223372036854775808,
      "title": "id",
      "type": "integer"
    },
    "some_field": {
      "description": "Overridden Name",
      "maxLength": 100,
      "minLength": 1,
      "title": "Custom Title",
      "type": "string"
    },
    "related_field": {
      "description": "Verbose",
      "title": "Verbose",
      "type": "integer",
      "x-ngc-extension": "ngc"
    }
  },
  "required": [
    "id",
    "some_field",
    "related_field"
  ],
  "title": "SomeModelSchema",
  "type": "object"
}

And as we can assert the InferExcept behavior, the generated schema will reflect the overridden properties as specified in the Pydantic model.

In [35]:
schema = SomeSchema.model_json_schema()
# Assert the title is as expected
assert schema["properties"]["some_field"]["title"] == CUSTOM_TITLE

# Assert the description is as expected
assert schema["properties"]["some_field"]["description"] == CUSTOM_DESCRIPTION

# Assert the json_schema_extra is as expected
assert schema["properties"]["related_field"][CUSTOM_SCHEMA_EXT] == CUSTOM_SCHEMA_VALUE